In [ ]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

test_df = pd.read_csv('dataset_preprocessing.csv', index_col=0)
label = test_df["Survived"]
test_df = test_df.drop(["Survived"], axis=1)
test_df.head()

### 1. Staging Model 로드 및 예측

In [ ]:
# To do
# load staging model

model_uri = "models:/hellomlflow/Staging"
loaded_model = mlflow.sklearn.load_model(model_uri)

In [ ]:
loaded_model

In [ ]:
y_prd = loaded_model.predict(pd.DataFrame(test_df))
accuracy_score(y_prd, label)

### 2. Production Model 로드 및 예측

In [ ]:
# To do
# load production model
model_uri = "models:/hellomlflow/Production"
loaded_model = mlflow.sklearn.load_model(model_uri)

In [ ]:
loaded_model

In [ ]:
y_prd = loaded_model.predict(pd.DataFrame(test_df))
accuracy_score(y_prd, label)

# 기존의 production accuracy : 0.920314253647587

In [ ]:
test_df.iloc[0].values

### 3. MLFlow 를 이용한 API Serving 

In [ ]:
mlflow models serve -m ./mlartifacts/899358510632907849/d20dbd4fbf2a421eab08dadf49bc8e36/artifacts/model -p 5001 --no-conda


In [ ]:
import pandas as pd
import requests

# Define local host and endpoint url
host = "127.0.0.1"
url = f"http://{host}:5001/invocations"

# Create dictionary with pandas DataFrame in the split orientation
json_data = {"dataframe_split": test_df[:10].to_dict(orient="split")}

# Score model
response = requests.post(url, json=json_data)
print(f"\nPyfunc 'predict_interval':\n${response.json()}")


In [ ]:
# Create dictionary with pandas DataFrame in the split orientation
json_data = {"dataframe_split": test_df[:20].to_dict(orient="split")}

# Score model
response = requests.post(url, json=json_data)


In [ ]:

print(f"\nPyfunc 'predict_interval':\n${response.json()}")

### 4. Cloud 배포 

##### AWS Sagemaker
* mlflow sagemaker build-and-push-container
* mlflow sagemaker deploy {parameters}

##### AzureML
* mlflow azureml export -m {model' path} -o test-output